<a href="https://colab.research.google.com/github/Golem8/Music-Genre-Classifier/blob/main/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#import libraries
!pip install pydub
import pydub
import librosa
import os
from pydub import AudioSegment 
from pydub.utils import make_chunks

from pydub import AudioSegment
from pydub.generators import WhiteNoise
import numpy as np
import soundfile

In [4]:
def augment_export(outputPath,iteration,chunk):
  # augment using white noise and pitch
  # adapted from https://stackoverflow.com/questions/56023174/how-to-add-noise-to-audiosegment-using-pydub

  filepathwn = outputPath[:-4] + "_" + str(iteration) + "_wn.wav"
  filepathreg = outputPath[:-4] + "_" + str(iteration) + "_reg.wav"

  #-20 lowers the dB volume of the white noise
  whitenoise = WhiteNoise().to_audio_segment(duration=len(chunk),volume=-20)
  chunkWithNoise = chunk.overlay(whitenoise)
  
  # Debugging
  #print ("exporting", filepathwn, "and",filepathreg)

  #export the varients as .wav files (for easy use in librosa)
  chunkWithNoise.export(filepathwn, format="wav")
  chunk.export(filepathreg, format="wav")

  #issue with pitch shift, can add later
  """
  # https://stackoverflow.com/questions/58810035/converting-audio-files-between-pydub-and-librosa
  samples = chunk.get_array_of_samples()
  arr = np.array(samples).astype(np.float32)

  #https://groups.google.com/g/librosa/c/XWae4PdbXuk
  filepathpitch = outputPath[:-4] + "_" + str(i) + "_pitch.wav"
  # sampling rate for our dataset is 22050
  # https://www.researchgate.net/publication/311458869_FMA_A_Dataset_For_Music_Analysis
  sr=22050
  y_shifted = librosa.effects.pitch_shift(arr, sr, n_steps=1) # shifted by 4 half steps
  soundfile.write(filepathpitch, y_shifted, sr, subtype=None, endian=None, format=None, closefd=True)
  """


In [5]:
data_path = "/content/drive/MyDrive/APS360 Team Project/data/testsmall"

# recursively goes though subdirectores and files
# adapted from https://newbedev.com/how-to-iterate-over-files-in-a-given-directory
for current_path, dirs_in_path, files_in_path in os.walk(data_path):
    # dont need to use dirs_in_path or directories, only interested in files
    for file in files_in_path:
        filepath = current_path + '/' + file

        # only keep the path if it is a music file, exclude 
        # other files in dataset
        if filepath.endswith(".mp3"): # then a music file has been found
            print("working on file",filepath)
            #https://stackoverflow.com/questions/65857983/how-can-i-split-an-audio-file-into-multiple-audio-wav-files-from-folder

            audio_object = AudioSegment.from_mp3(filepath) 
            chunk_length_ms = 2000 #2000 ms clips each 

            #slices the audio into 2 second clips
            chunks = make_chunks(audio_object,chunk_length_ms)
            for iteration, chunk in enumerate(chunks): 
              outputPath = data_path + "/output/" + file[:-4] + ".wav"

              # if the chunks are not 2000 ms long (the last one may be shorter)
              #discard them as they would
              # mess up the creation of the spectrograms
              if len(chunk) == 2000:
                # will export the name with scheme path_iteration_(augment_type).wav
                augment_export(outputPath,iteration,chunk)
              else:
                print("deleted malformed chunk of length ", len(chunk),"ms")



        
              


working on file /content/drive/MyDrive/APS360 Team Project/data/testsmall/000/000010.mp3
deleted malformed chunk of length  1977 ms
working on file /content/drive/MyDrive/APS360 Team Project/data/testsmall/000/000005.mp3
deleted malformed chunk of length  3 ms
working on file /content/drive/MyDrive/APS360 Team Project/data/testsmall/000/000002.mp3
deleted malformed chunk of length  1977 ms
working on file /content/drive/MyDrive/APS360 Team Project/data/testsmall/001/001040.mp3
deleted malformed chunk of length  1977 ms
working on file /content/drive/MyDrive/APS360 Team Project/data/testsmall/001/001069.mp3
deleted malformed chunk of length  1977 ms
working on file /content/drive/MyDrive/APS360 Team Project/data/testsmall/001/001066.mp3
deleted malformed chunk of length  3 ms
working on file /content/drive/MyDrive/APS360 Team Project/data/testsmall/001/001039.mp3
deleted malformed chunk of length  1977 ms
